A script to scrape the Court Proceedings database of EPA Victoria, which goes back to 2000.

In [1]:
## Libraries
import requests
from bs4 import BeautifulSoup, Tag
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd
import networkx as nx
import math
import urllib
import seaborn as sns
import datetime

from requests_html import AsyncHTMLSession
from requests_html import HTMLSession

In [2]:
now = datetime.datetime.now()
current_year, current_month, current_day = now.year, now.month, now.day

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
df = pd.DataFrame(columns=['Year', 'URL'])

def getLinks(year):
    
    url = f'https://www.epa.vic.gov.au/about-epa/public-registers/court-proceedings?m=&y={year}&ps=10000'

    asession = AsyncHTMLSession()
    session = HTMLSession()
    r = session.get(url)
    
    async def get_pythonorg():
        r = await asession.get(url)
        
    for link in r.html.absolute_links:
        #if 'search-prosecutions' in link:  
        if 'court-proceedings/' in link: 
        
            n = df.shape[0]
            df.loc[n,'Year'] = year
            df.loc[n,'URL'] = link
            df.loc[n,'Stub'] = link.split('/')[-1]
            
            # get data from individual pages
            url2 = link
            r2 = session.get(url2)
            
            offender = r2.html.find(".content-header")
            offender = offender[0].text
            df.loc[n, 'Offender'] = offender
                        
            dt = r2.html.find('dt')
            dd = r2.html.find("dd")
        
            for number, item in enumerate(dd):
                column = dt[number].text
                value = item.text
                value = value.replace(',','~').replace('\n',' ')
                df.loc[n, column] = value
                
            court_orders_made = r2.html.find(".cm-rich-text")
            for item in court_orders_made:
                item = item.text
                item = item.replace('\n',' ').replace('Court orders made',' ').replace('$','\$')
                item = item.lstrip()
                df.loc[n, 'Court orders made'] = item

In [5]:
for year in range(2000,current_year+1):
    print(year, end = ' ')
    getLinks(year)

2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 

In [6]:
# calculate costs of fines etc - note that it is also picking up other mentions of $ terms

df['Total ($)'] = df['Court orders made'].str.replace('\$ ', '$').str.replace(',', '')
df['Total ($)'] = df['Total ($)'].str.findall(r'\$([0-9,.]+[0-9]+)')

df['Payments mentioned'] = df['Total ($)'].copy().apply(lambda x: [float(y) for y in  x])

df['Total ($)'] = df['Payments mentioned'].apply(lambda x: sum(x))

<ipython-input-6-fdbaba323adf>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Total ($)'] = df['Court orders made'].str.replace('\$ ', '$').str.replace(',', '')


In [7]:
# for helping to weed out payments where the payment is repeated in text

'''for index, row in df.iterrows():
    a = len(row['Payments mentioned'])
    b = len(set(row['Payments mentioned']))
    if a == b:
        df.loc[index, 'Potential payment double up'] = False
    else:
        df.loc[index, 'Potential payment double up'] = True
        
df.loc[df['Potential payment double up'] == True]

df = df.drop(columns=['Potential payment double up'])
'''

# these are some of the anolamlies I have found and fixed
df.loc[df['Stub'] == 'shell-refining-australia-pty-ltd_2', 'Total ($)'] = (75000.0 + 60000.0)
df.loc[df['Stub'] == 'betta-foods-australia-pty-ltd', 'Total ($)'] = 65459.15
df.loc[df['Stub'] == 'scalzo-trading-co-proprietary-limited', 'Total ($)'] = 65000.0
df.loc[df['Stub'] == 'universal-metals-pty-ltd', 'Total ($)'] = (45000.0 + 18372.19)
df.loc[df['Stub'] == 'mobil-refining-australia-pty-ltd_1', 'Total ($)'] = 350000.00
df.loc[df['Stub'] == 'mobil-refining-australia-pty-ltd_2', 'Total ($)'] = (150000.0 + 135000.0)
df.loc[df['Stub'] == 'willis-and-palmer-vic-pty-ltd', 'Total ($)'] =  (5000.0 + 951.0)
df.loc[df['Stub'] == 'west-point-vic-pty-ltd_1', 'Total ($)'] = 190000.0
df.loc[df['Stub'] == 'mccormacks-demolition-pty-ltd', 'Total ($)'] = 20200.0
df.loc[df['Stub'] == 'casey-edward-mccormack', 'Total ($)'] = 50200
df.loc[df['Stub'] == 'betta-foods-australia-pty-ltd', 'Total ($)'] = (150000.0 + 135000.0)
df.loc[df['Stub'] == 'teekay-navion-offshore-loading-pte-ltd', 'Total ($)'] = 200000.0
df.loc[df['Stub'] == 'fcl-interstate-transport-service-pty-ltd', 'Total ($)'] = (75000.0 + 14895.0)
df.loc[df['Stub'] == 'golden-circle-limited', 'Total ($)'] = (50000.0  + 5500.0)
df.loc[df['Stub'] == 'nuplex-industries-aust-pty-ltd', 'Total ($)'] = (160000.0 + 13464.0)
df.loc[df['Stub'] == 'transpacific-industries-pty-ltd', 'Total ($)'] = (80000.0 + 10000.0)
df.loc[df['Stub'] == 'hyde-park-tank-depot-pty-ltd_1', 'Total ($)'] = (40000.0 + 13825.3)    
df.loc[df['Stub'] == 'australian-tallow-producers-pty-ltd', 'Total ($)'] = 200000.0 
df.loc[df['Stub'] == 'tasman-group-services-pty-ltd', 'Total ($)'] = 230000.0
df.loc[df['Stub'] == 'copy-of-160-leicester-pty-ltd', 'Total ($)'] = (30000.0 + 35000.0) 

In [8]:
# isolate the offences committed

df['section'] = df['Type of offence']

df['section'] = df['section'].str.replace(r'Charges 2~ 4 and 6 struckout.', '', regex = True) #special case

df['section'] = df['section'].str.replace(r'Section', 'section', regex = True)
df['section'] = df['section'].str.replace(r'Charge: ', 'Charges ', regex = True)
df['section'] = df['section'].str.replace(r'Charges: ', 'Charges ', regex = True)
df['section'] = df['section'].str.replace(r': (?=\d)', ' section', regex = True)
df['section'] = df['section'].str.replace(r' s(?=\d)', ' section', regex = True)
df['section'] = df['section'].str.replace(r' s\.(?=\d)', ' section', regex = True)
df['section'] = df['section'].str.replace(r' s\. (?=\d)', ' section', regex = True)
df['section'] = df['section'].str.replace(r' section (?=\d)', ' section', regex = True)
df['section'] = df['section'].str.replace(r' sections (?=\d)', ' section', regex = True)

df['section'] = df['section'].str.replace(r' February ', '.3.', regex = True)
df['section'] = df['section'].str.replace(r' March ', '.3.', regex = True)
df['section'] = df['section'].str.replace(r'\d{1,2}th', '', regex = True)   #7th and 21st
df['section'] = df['section'].str.replace(r'\d{1,2}st', '', regex = True)  
df['section'] = df['section'].str.replace(r'\d{1,2}.\d{1,2}.\d{2,4} and \d{1,2}.\d{1,2}.\d{2,4}~', 'DATE', regex = True)     # dates eg 01.05.99 and 09.08.99~
df['section'] = df['section'].str.replace(r'\d{1,2}.\d{1,2}.\d{2,4} to \d{1,2}.\d{1,2}.\d{2,4}~', 'DATE', regex = True)     # dates eg 01.05.99 and 09.08.99~

df['section'] = df['section'].str.replace(r' and (?=\d)', ' section', regex = True)
df['subsection'] = df['section'].str.findall(r'(?<=section)([\w\()]+)')
df['section'] = df['section'].str.findall(r'(?<=section)([\w]+)')

In [9]:
# With/Out Conviction

for index, row in df.iterrows():
    if 'with conviction' in str(row['Court orders made']).lower():
        df.loc[index, 'Conviction'] = 'With'
    elif 'without conviction' in str(row['Court orders made']).lower():
        df.loc[index, 'Conviction'] = 'Without'
    else:
        df.loc[index, 'Conviction'] = None

In [10]:
df.to_csv(f'CSVs/EPA_Victoria_Court_Proceedings_2000_to_{current_year}{current_month}{current_day}.csv', index=False)

df

,Year,URL,Stub,Offender,Type of offence,Date of court hearing,Date of court order,Court magistrate,Court location,Court orders made,Date of offence,Proceeding number,Background of offense,Remedial action taken by EPA,Reasons for prosecution,Total ($),Payments mentioned,section,subsection,Conviction
0,2000,https://www.epa.vic.gov.au/about-epa/public-registers/court-proceedings/pj-and-iw-gillespie-pty-ltd,pj-and-iw-gillespie-pty-ltd,PJ & IW Gillespie Pty Ltd,Dump industrial waste at a site not licensed to accept industrial waste contrary to section 27A(2)(a) of the Environment Protection Act 1970,15 September 2000,15 September 2000,Magistrate Cashmore,Bacchus Marsh,"Without conviction, fined \$2,000 Costs: \$ 1006.50",NaN,NaN,NaN,NaN,NaN,3006.50,"[2000.0, 1006.5]",[27A],[27A(2)(a)],Without
1,2000,https://www.epa.vic.gov.au/about-epa/public-registers/court-proceedings/conmor-plant-hire-pty-ltd,conmor-plant-hire-pty-ltd,Conmor Plant Hire Pty Ltd,1. Pollution of waters so that the condition of the waters was so changed as to make or be reasonably expected to make those waters poisonous~ harmful or potentially harmful to animals~ birds~ wildlife~ fish aquatic life: 39(1)(c) 2. Polluting the atmosphere so that the condition of the atmosphere was so changed as to make it detrimental to any beneficial use made of the atmosphere: 41(1)(e),13 September 2000,13 September 2000,Magistrate Cashmore,Sunshine,"Without conviction, fined \$8,000 Costs: \$ 6,861.52",NaN,NaN,NaN,NaN,NaN,14861.52,"[8000.0, 6861.52]","[39, 41]","[39(1)(c), 41(1)(e)]",Without
2,2000,https://www.epa.vic.gov.au/about-epa/public-registers/court-proceedings/chilean-club-of-victoria,chilean-club-of-victoria,Chilean Club of Victoria,Compensation order pursuant to s65A further to charges found proven on 11 December 1997.,10 July 2000,10 July 2000,Magistrate A. Spillane,Broadmeadows,"Order for compensation of \$79,833. Costs: \$4,225.00",NaN,NaN,NaN,NaN,NaN,84058.00,"[79833.0, 4225.0]",[65A],[65A],None
3,2000,https://www.epa.vic.gov.au/about-epa/public-registers/court-proceedings/calleja-properties-pty-ltd,calleja-properties-pty-ltd,Calleja Properties Pty Ltd,Disobeyed Conditions 6(d)~ 6(I) & 9 of Licence in contravention of s. 27(2) of the Environment Protection Act 1970.,24 October 2000,24 October 2000,Magistrate Goldberg,Broadmeadows,"Without conviction, fined \$7,500 Costs: \$4,682",NaN,NaN,NaN,NaN,NaN,12182.00,"[7500.0, 4682.0]",[27],[27(2)],Without
4,2000,https://www.epa.vic.gov.au/about-epa/public-registers/court-proceedings/captain-terrence-william-judge,captain-terrence-william-judge,Captain Terrence William Judge (Master of MV Helix),Polluting waters so that the condition of those waters was so changed as to make those waters detrimental to a beneficial use made of those waters: s. 39(1)(e),30 October 2000,30 October 2000,Magistrate Purcell,Geelong,"Without conviction, fined \$5,000 Costs: \$5,497.00",NaN,NaN,NaN,NaN,NaN,10497.00,"[5000.0, 5497.0]",[39],[39(1)(e)],Without
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,2021,https://www.epa.vic.gov.au/about-epa/public-registers/court-proceedings/ibrahim,ibrahim,"ALI IBRAHIM, Ibrahim",Charge: 1. Application under Section 64 of the Environment Protection Act 1970 for enforcement of Court orders requiring the removal of all industrial waste from the premises located at 320 Bacchus Marsh Road~ Corio~ made 29 July 2019.,18 January 2021,18 January 2021,F J Holzer,Geelong,"1. Application Granted: Order that the Accused be fined for his contempt and non-compliance of my orders of 29 July 2019. A fine of \$40,000 with conviction, together with legal costs of \$9,886.25, such sums to be paid on or before 17 May 2021.",On or about 23 July 2019,J13100088,On 29 July 2019~ Ibrahim Ali Ibrahim (the accused) was found guilty of a number of offences under the EP Act~ including permitting the deposit of industrial waste~ in relation to the premises located at 320 Bacchus Marsh Road~ Corio. The 